In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_education(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    section = soup.find("div", id="education")
    data = []

    if section:
        tables = section.find_all("table")
        for table in tables:
            rows = table.find_all("tr")
            for row in rows:
                cells = row.find_all(["th", "td"])
                row_data = [cell.get_text(strip=True) for cell in cells]
                if row_data:
                    data.append(row_data)

    if not data:
        print("❌ Aucun tableau trouvé ou structure vide")
        return None

    # Première ligne = noms de colonnes
    colonnes = data[0]
    valeurs = [row for row in data[1:] if len(row) == len(colonnes)]  # 🔥 sécurise les lignes valides

    # Création du DataFrame
    df_education = pd.DataFrame(valeurs, columns=colonnes)

    # Remplacer les cellules vides par 0
    df_education = df_education.replace("", 0)

    return df_education

In [32]:
# Lire par chunks
chunksize = 50
reader = pd.read_csv("communes-france-2025.csv", dtype={"code_insee": str}, low_memory=False, chunksize=chunksize)

data_concat = []

for chunk_index, df_chunk in enumerate(reader):
    # Supprimer première colonne si elle est inutile
    if df_chunk.columns[0].startswith("Unnamed"):
        df_chunk = df_chunk.drop(df_chunk.columns[0], axis=1)

    # Filtrer les villes > 20k habitants
    filtered_source = df_chunk[df_chunk["population"] >= 20000]

    for i in range(filtered_source.shape[0]):
        code_insee = filtered_source["code_insee"].iloc[i]
        ville = filtered_source["nom_sans_accent"].iloc[i]

        url = f"https://www.linternaute.com/ville/{ville}/ville-{code_insee}/education"
        recherche = get_education(url)
        if recherche is not None:
            recherche["code_insee"] = code_insee
            recherche["Ville"] = ville
            data_concat.append(recherche)

            print("✅ Ligne ajoutée au DataFrame :")
            print(f"{ville} : {code_insee}")
            print("------------------------------------------------------------")

print("Recherches terminées")
# Concat final
df_final = pd.concat(data_concat, ignore_index=True)

scolarises = [
    "2 - 5 ans", "6 - 10 ans", 
    "11 - 14 ans", "15 - 17 ans",
    "18 - 24 ans", "25 - 29 ans",
    "30 ans et plus"
]

non_scolarises = [
    "Aucun diplôme", "Brevet des collèges", "CAP / BEP",
    "Baccalauréat / brevet professionnel", "De Bac +2 à Bac +4", "Bac +5 et plus",
]

# On initialise la colonne
df_final["type"] = df_final.iloc[:, 0].apply(
    lambda x: "Scolarisés" if x in scolarises else "Diplome"
)

✅ Ligne ajoutée au DataFrame :
bourg-en-bresse : 01053
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
oyonnax : 01283
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
laon : 02408
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
saint-quentin : 02691
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
soissons : 02722
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
montlucon : 03185
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
vichy : 03310
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
manosque : 04112
------------------------------------------------------------
✅ Ligne ajoutée au DataFrame :
gap : 05061
------------------------------------------------------------
✅ Ligne ajoutée au 

In [33]:
df_final = df_final[df_final["Données 2021"] != "Données 2021"]
df_final.to_csv("education1.csv", index=False, encoding="utf-8-sig", sep=";")

In [ ]:
df_ecole = pd.read_csv("education1.csv")


In [38]:
df_ecole = pd.read_csv("education.csv")

# Liste des types d'établissements scolaires
types_etablissements = [
    "Ecoles maternelles",
    "Ecoles élémentaires",
    "Collèges",
    "Lycées généraux",
    "Lycées professionnels",
    "Lycées agricoles",
    "Etablissements avec classes préparatoires aux grandes écoles",
    "Ecoles de formation sanitaire et sociale",
    "Ecoles de commerce, gestion, administration d'entreprises, comptabilité, vente",
    "Unités de formation et de recherche (UFR)",
    "Instituts universitaires (IUP, IUT et IUFM)",
    "Ecoles d'ingénieurs",
    "Etablissements de formation aux métiers du sport",
    "Centres de formation d'apprentis (hors agricoles)",
    "Centres de formation d'apprentis agricoles",
    "Autres écoles d'enseignement supérieur",
    "Autres formations post-bac non universitaire"
]

# Suppose que ton DataFrame s'appelle df et que la colonne à filtrer s'appelle 'Label'
df_filtered = df_ecole[df_ecole['Données 2021'].isin(types_etablissements)]

df_filtered = df_filtered.iloc[:, :-1]

df_filtered.to_csv("ecole.csv", index=False, encoding="utf-8-sig", sep=";")

In [40]:
dff = pd.read_csv("ecole.csv", sep=";")
dff

,Données 2021,Personnes scolarisées,code_insee,Ville
0,Ecoles maternelles,2,01053,bourg-en-bresse
1,Ecoles élémentaires,21,01053,bourg-en-bresse
2,Collèges,7,01053,bourg-en-bresse
3,Lycées généraux,4,01053,bourg-en-bresse
4,Lycées professionnels,4,01053,bourg-en-bresse
...,...,...,...,...
8172,Etablissements de formation aux métiers du sport,0,97422,le-tampon
8173,Centres de formation d'apprentis (hors agricoles),13,97422,le-tampon
8174,Centres de formation d'apprentis agricoles,1,97422,le-tampon
8175,Autres écoles d'enseignement supérieur,0,97422,le-tampon


In [42]:
dfff = pd.read_csv("education1.csv", sep=";")
dfff

,Données 2021,Personnes scolarisées,% de la population scolarisée,Moyenne des villes,code_insee,Ville,Données nc,type
0,2 - 5 ans,1 235,"13,5 %","15,6 %",01053,bourg-en-bresse,NaN,Scolarisés
1,6 - 10 ans,2 228,"24,3 %","29,2 %",01053,bourg-en-bresse,NaN,Scolarisés
2,11 - 14 ans,1 764,"19,3 %","24,9 %",01053,bourg-en-bresse,NaN,Scolarisés
3,15 - 17 ans,1 421,"15,5 %","18,4 %",01053,bourg-en-bresse,NaN,Scolarisés
4,18 - 24 ans,2 117,"23,1 %","9,1 %",01053,bourg-en-bresse,NaN,Scolarisés
...,...,...,...,...,...,...,...,...
6276,NaN,nc,nc,"5,5 %",97611,mamoudzou,Brevet des collèges,Non scolarisés
6277,NaN,nc,nc,"29,3 %",97611,mamoudzou,CAP / BEP,Non scolarisés
6278,NaN,nc,nc,"18,8 %",97611,mamoudzou,Baccalauréat / brevet professionnel,Non scolarisés
6279,NaN,nc,nc,"19,8 %",97611,mamoudzou,De Bac +2 à Bac +4,Non scolarisés


In [52]:
dfff = dfff[dfff["Données 2021"].notna()]
dfff = dfff.drop(columns="Données nc")

In [53]:
dfff

,Données 2021,Personnes scolarisées,% de la population scolarisée,Moyenne des villes,code_insee,Ville,type
0,2 - 5 ans,1 235,"13,5 %","15,6 %",01053,bourg-en-bresse,Scolarisés
1,6 - 10 ans,2 228,"24,3 %","29,2 %",01053,bourg-en-bresse,Scolarisés
2,11 - 14 ans,1 764,"19,3 %","24,9 %",01053,bourg-en-bresse,Scolarisés
3,15 - 17 ans,1 421,"15,5 %","18,4 %",01053,bourg-en-bresse,Scolarisés
4,18 - 24 ans,2 117,"23,1 %","9,1 %",01053,bourg-en-bresse,Scolarisés
...,...,...,...,...,...,...,...
6248,Brevet des collèges,3 276,"5,8 %","5,5 %",97422,le-tampon,Non scolarisés
6249,CAP / BEP,11 561,"20,6 %","29,3 %",97422,le-tampon,Non scolarisés
6250,Baccalauréat / brevet professionnel,9 915,"17,7 %","18,8 %",97422,le-tampon,Non scolarisés
6251,De Bac +2 à Bac +4,8 338,"14,9 %","19,8 %",97422,le-tampon,Non scolarisés


In [54]:
dfff.to_csv("education1.csv", index=False, encoding="utf-8-sig", sep=";")